# Is there any relationship between suicides and forest area?

Required tools and libraries in this analysis:

In [12]:
import pandas as pd
import basedosdados as bd
from matplotlib import pyplot as plt

%matplotlib inline

## Available data
### To approach this question we'll consider the following datasets:
#### 1. [Sistema de Informações sobre Mortalidade (SIM)](https://basedosdados.org/dataset/br-ms-sim?bdm_table=microdados)
CID-10 codes of interest: X60-X84 e Y87.0 

#### 2. [Censo Agropecuário](https://basedosdados.org/dataset/br-ibge-censo-agropecuario?bdm_table=municipio)


#### 3. [População brasileira](https://basedosdados.org/dataset/br-ms-populacao?bdm_table=municipio)


### References:
- [1] [Mortalidade por suicídio e notificações de lesões autoprovocadas no Brasil](https://www.gov.br/saude/pt-br/centrais-de-conteudo/publicacoes/boletins/epidemiologicos/edicoes/2021/boletim_epidemiologico_svs_33_final.pdf)

## Simplest case: municipios, 2007 
At first, lets focus on the year of 2017, as it is covered by all datasets and the most recent available Censo Agropecuário.

In [27]:
class Get_data:
    def _unpack(self, l):
        return ", ".join(map(str, l))

    def _get_data(self, filename, query):
        try:
            return pd.read_csv(f'../data/{filename}.csv')
        except FileNotFoundError:
            df = bd.read_sql(query=query,
                             billing_project_id='explorando-basedosdados')
            df.to_csv(
                f'../data/{filename}.csv', index=False)
            return df

    def sim_suic_municipio_2017(self):
        filename = 'sim_suic_municipio_2017'
        columns = ('id_municipio', 'SUM(numero_obitos) as numero_obitos')
        query = f"""
                SELECT 
                    {self._unpack(columns)}
                FROM 
                    br_ms_sim.municipio_causa
                WHERE 
                    ano = 2017
                    AND (
                        CONTAINS_SUBSTR(causa_basica, 'X6')
                        OR CONTAINS_SUBSTR(causa_basica, 'X7')
                        OR CONTAINS_SUBSTR(causa_basica, 'X8')
                        OR causa_basica = 'Y870'
                        )
                    GROUP BY id_municipio
                """
        return self._get_data(filename, query)

    def agro_forest_municipio_2017(self):
        filename = 'agro_forest_municipio_2017'
        columns = ('id_municipio',
                   'area_total',
                   'area_mata_natural',
                   'area_mata_plantada',
                   'area_sistema_agroflorestal',
                   'area_mata')
        query = f"""
                SELECT 
                    {self._unpack(columns)}
                FROM 
                    br_ibge_censo_agropecuario.municipio
                WHERE
                    ano = 2017
                    """
        return self._get_data(filename, query)

    def sim_suic_pop_2017(self):
        query = """
                WITH obitos AS 
                    (
                        SELECT 
                            id_municipio, SUM(numero_obitos) as numero_obitos 
                        FROM 
                            basedosdados.br_ms_sim.municipio_causa
                        WHERE 
                            ano = 2017 
                        AND (
                            CONTAINS_SUBSTR(causa_basica, 'X6')
                            OR CONTAINS_SUBSTR(causa_basica, 'X7')
                            OR CONTAINS_SUBSTR(causa_basica, 'X8')
                            OR causa_basica = 'Y870'
                            )
                        GROUP BY 
                            id_municipio
                        LIMIT 
                            500
                    ), 
                    populacao AS 
                        (
                            SELECT 
                                id_municipio, populacao 
                            FROM 
                                `basedosdados.br_ibge_populacao.municipio` 
                            WHERE 
                                ano = 2017 
                            LIMIT 
                                500
                        )
                SELECT 
                obitos.id_municipio, obitos.numero_obitos, populacao.populacao 
                FROM 
                obitos JOIN populacao ON obitos.id_municipio = populacao.id_municipio
                """
        return self._get_data('sim_sui_pop_municipio_2017',query)
        


get_data = Get_data()
sim_suic_2017 = get_data.sim_suic_municipio_2017()
agro_forest_2017 = get_data.agro_forest_municipio_2017()
sim_sui_pop_2017 = get_data.sim_suic_pop_2017()


Downloading: 100%|██████████| 62/62 [00:00<00:00, 137.70rows/s]


In [29]:
sim_sui_pop_2017

,id_municipio,numero_obitos,populacao
0,1200203,6,82622
1,1200252,2,17340
2,1200302,4,32360
3,1200500,4,43139
4,1200609,4,40024
...,...,...,...
57,2102077,1,16214
58,2102200,2,28306
59,2102325,1,71979
60,2102408,1,11019


In [22]:
agro_forest_2017

,id_municipio,area_total,area_mata_natural,area_mata_plantada,area_sistema_agroflorestal,area_mata
0,1100015,372746.0,108944.0,258.0,10481.0,109202.0
1,1100023,334256.0,88865.0,615.0,1573.0,89480.0
2,1100031,113085.0,17271.0,593.0,0.0,17864.0
3,1100049,221390.0,31620.0,180.0,0.0,31800.0
4,1100056,126686.0,27954.0,0.0,489.0,27954.0
...,...,...,...,...,...,...
5558,5222005,79967.0,12186.0,715.0,0.0,12901.0
5559,5222054,62341.0,5304.0,0.0,2626.0,5304.0
5560,5222203,84770.0,5542.0,0.0,4663.0,5542.0
5561,5222302,186118.0,34659.0,724.0,1083.0,35383.0


In [30]:
df = pd.merge(sim_sui_pop_2017,agro_forest_municipio_2017,how='left',on='id_municipio')

,id_municipio,numero_obitos,populacao,area_total,area_mata_natural,area_mata_plantada,area_sistema_agroflorestal,area_mata
0,1200203,6,82622,132413.0,89938.0,0.0,5297.0,89938.0
1,1200252,2,17340,172858.0,105137.0,34.0,88.0,105171.0
2,1200302,4,32360,346175.0,248128.0,0.0,8357.0,248128.0
3,1200500,4,43139,378724.0,234667.0,26.0,543.0,234693.0
4,1200609,4,40024,377807.0,260697.0,84.0,15299.0,260781.0
...,...,...,...,...,...,...,...,...
57,2102077,1,16214,33387.0,1505.0,0.0,1412.0,1505.0
58,2102200,2,28306,41712.0,8606.0,0.0,4423.0,8606.0
59,2102325,1,71979,100739.0,10608.0,0.0,0.0,10608.0
60,2102408,1,11019,10395.0,100.0,0.0,397.0,100.0
